<a href="https://colab.research.google.com/github/taichi0315/neural-network/blob/master/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E5%9F%BA%E7%A4%8E%EF%BC%88%E3%82%B3%E3%83%BC%E3%83%89%E6%9C%AA%E5%AE%8C%E6%88%90%E7%89%88%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目的
ニューラルネットワークとはどのような構造であるか、irisデータセットを用いた花の分類を行うニューラルネットワークを実装することで、ニューラルネットワークの基礎を理解することを目的とします。

# 本テキストの対象者
- pythonを使って条件分岐・繰り返し処理・関数の作成を行うことができる。
- 線形代数の基礎知識がある。

# 実装を行う例題

本テキストでは、irisデータセットを用いてニューラルネットワークの実装を行います。irisデータセットは、一つのサンプルにつき「ガクの長さ」、「ガクの幅」、「花弁の長さ」、「花弁の幅」の4つの変数とそのサンプルが何の花のサンプルであるかの正解ラベルがセットで与えられます。今回は簡単のため、「ガクの長さ」、「花弁の長さ」の２つの変数が与えられた場合に、その花の種類が何であるかの分類を行うニューラルネットワークを実装します。

# データの準備・説明

## irisデータセットのダウンロード
irisデータセットのダウンロードはpythonの機械学習ライブラリであるscikit-learnを用いて行います。scikit-learnについて本テキストではこれ以上触れません。

In [0]:
from sklearn.datasets import load_iris
iris = load_iris()

## 分類を行う花の種類
今回は、花の種類がsetosaとversicolorのどちらであるかの分類を行います。irisデータセットでは花の種類の区別が整数値で扱われているため、以下の辞書を作成し、花の種類の可視化に役立てます。0がsetosa、1がversicolorに対応します。

In [0]:
iris_name = {0:'setosa', 1:'versicolor'}

## サンプルの抽出
以下のコードによって、irisデータセット１つのサンプルにつき「ガクの長さ」、「花弁の長さ」の２つの変数とそのサンプルが何の花のサンプルであるかの正解ラベルを抽出します。setosaとversicolorそれぞれ50個ずつのサンプルが得られます。

In [0]:
X = []
Y = []

for i in range(100):
    X.append([iris.data[i][0], iris.data[i][2]])
    Y.append(iris_name[iris.target[i]])

以下のコードで、抽出したサンプルの先頭５つを確認します。

In [0]:
print('[ガクの長さ,花弁の長さ] 花の種類')
for i in range(5):
    print(X[i],Y[i])

[ガクの長さ,花弁の長さ] 花の種類
[5.1, 1.4] setosa
[4.9, 1.4] setosa
[4.7, 1.3] setosa
[4.6, 1.5] setosa
[5.0, 1.4] setosa


# ニューラルネットワークの基礎理論



## ニューラルネットワークとは
ニューラルネットワークとは、人間の脳を構成しているニューロン（神経細胞）を数理モデル化しネットワークとして組み合わせたものです。ニューロンを数理モデル化すると、与えられた入力に対して出力を返す関数になります。近年の機械学習ブームはニューラルネットワークの発展が引き金となっています。

<div align=center>
![代替テキスト](http://hokuts.com/wp-content/uploads/2015/11/perceptron.png)<br>
図5.1　ニューロンの概要図</div><br>

今回構築するニューラルネットワークは以下の図5.2のような構造になっています。

<div align=center>

![今回構築するニューラルネットワーク](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550165905.png)<br>

図5.2　３層ニューラルネットワーク</div><br>


ニューラルネットワークは、入力層・隠れ層・出力層の３つに分けられます。


- 入力層  
入力データが与えられる層。今回は$x_1=$ガクの長さ、$x_2=$花弁の長さとなる。

- 隠れ層  
入力層と出力層の中間の層。任意の数の層を設定する（今回は2層）。各層には任意の数のニューロンを設定する。

- 出力層  
隠れ層から受け取った値を元に計算を行い、結果を出力する層。今回は、$y_1$がsetosa、$y_2$がversicolorに対応する。具体的には、仮に与えられたサンプルの花の種類がsetosaである確率が80％、versicolorである確率が20％と予測される場合、$y_1=0.8$、$y_2=0.2$という結果が出力される。

## 各ニューロンでの計算
最初に、各ニューロンでどのような計算が行われるかを簡単に説明します。

<div align=center>
![各ニューロンにおける計算](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550167256.png)<br>
図5.3　各ニューロンでの計算</div><br>




各ニューロンでは、与えられた入力$x_i$に対してそれぞれ重み$w_i$を掛けます。そして、それぞれ重みを掛けたものの総和にバイアス$b$を足すことで$a$という結果が得られ、$a$を活性化関数$h()$で変換することで出力$y$が求められます。図5.2の今回構築するニューラルネットワークでは、簡単のためバイアスは省略してあります。以下に各ニューロンで行われる計算を数式でまとめます。

<div align=center>
$$
\begin{align}
        a &= w_1 \times x_1 + w_2 \times x_2 + b \quad &&(5.1)\\
        y &= h(a) \quad &&(5.2)
\end{align}
$$
</div>

また、数式(5.1)はベクトルの内積を用いて表すこともできます。

$$
\begin{align}
        u & = w_1 \times x_1 + w_2 \times x_2 + b\\
               & = \left(x_1, x_2 \right) 
                      \left(
                        \begin{array}{c}
                            w_1 \\
                            w_2 
                            \end{array}
                          \right) + b \quad &&(5.3)
\end{align}
$$
</div>

後で説明しますが、ニューラルネットワークでは、数式(5.3)のベクトルの内積と同様に行列の内積を用いて計算が行われることが一般的となっています。

## 重みとバイアス
ここで数式(5.1)に用いられている重みとバイアスについて説明をします。活性化関数については後述します。

### 重み
重みとは、それぞれの入力値がどれだけ重要かを表す値です。

irisデータセットで例えると、setosaの花はガクの長さ$x_1$がとても長い傾向にあるとした場合、$x_1$が持つ値の情報は重要になります。そのため、もし仮に図２のニューロンの出力が、satosaの分類予測値である$y_1$にとって重要な入力となる場合、$x_1$へ掛け合わせる$w_1$の値が大きい方が予測が当たりやすいということになります。


### バイアス
次にバイアスです。ここでは、バイアスのイメージを掴むために単回帰分析([詳しくはこちら](https://bellcurve.jp/statistics/course/9700.html))の例を挙げます。
例えばirisのデータを使って、ガクの長さ$x_1$が入力として与えられた時に花弁の長さ$x_2$を予測する問題が与えられたとします。

<div align=center>
![代替テキスト](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550168930.png)<br>
図5.4　単回帰分析の例
</div><br>

図5.4のグラフは、横軸がガクの長さ、縦軸が花弁の長さで、事前に与えられたデータ5個（$x_1$と$x_2$の組み合わせでデータ1つ分）の散布図を表しています。

<br>

単回帰分析で予測を行う場合、花弁の長さの予測値を$y$とすると、予測を行う式は一次関数$y=ax_1+b$となります。単回帰分析はこの一次関数の傾き$a$（重み）と切片$b$（バイアス）の値を調整することで予測の誤差を小さくしていきます。

<br>
では、バイアスが0が与えられず$b=0$の場合はどうなるでしょう。図5.4のグラフにおいて、赤色の線はバイアス$b$が誤差の少ない値に調整済みの場合、緑色の線は$b=0$であるバイアスが与えられなかった場合です。比べてみると、明らかに赤色の線の方がデータの特徴を掴めていることがわかります。

<br>

ニューラルネットワークにおけるバイアスは単回帰分析における予測式の切片と似たような役割を持ちます。与えられた入力に重みを掛け合わせた値にバイアスを足すことで、予測の精度を上げることに役立ちます。

# ３層ニューラルネットワークの実装
本章では、図5.2で示した３層のニューラルネットワークをpythonを用いて実装していきます。ライブラリはnumpyと呼ばれる科学計算ライブラリを用います。なお今回は、ニューラルネットワークの基礎を理解できるようにするため、簡単にニューラルネットワークを実装することができるpytorch、tensorflow、keras、chainerといったライブラリは使用しません。

## numpyのインポート
以下のコードでnumpyのインポートを行います。

In [0]:
import numpy as np

## 記号の確認
ここでニューラルネットワークで説明する処理を理解するにあたり、$w_{12}^{(1)}$や$a_1^{(1)}$などの記号を導入します。



<div align=center>
![代替テキスト](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550171407.png)<br>
図6.1　重みの記号
</div><br>

図6.1に示すとおり、重みや隠れ層のニューロンの右上には「(1)」とつきます。これは第１層の重み・ニューロンという意味になります。重みの右下には数字が２つ並びます。次層のニューロンと前層のニューロンの順でインデックス番号が振られます。例として、$w_{12}^{(1)}$は前層２番目のニューロン$x_2$から次層１番目のニューロン$a_1^{(1)}$への重みであることを意味します。

## 入力層⇨第１層目
それでは、入力層から隠れ層である第１層目への伝達を実装します。入力層から第１層目の１番目のニューロン$a_1^{(1)}$への伝達を以下の図6.2で表します。

<div align=center>
![代替テキスト](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550173084.png)<br>
図6.2　入力層から第１層目の１番目のニューロン$a_1^{(1)}$への伝達
</div><br>

この図6.2を参考にし、ニューロン$a_1^{(1)}$を数式で表すと以下のようになります。

<div align=center>
$$
\begin{align}
        a_1^{(1)} &= w_{11}^{(1)} \times x_1 + w_{12}^{(1)} \times x_2 + b_1^{(1)} \quad &&(6.1)\\
\end{align}
$$
</div>

ニューロン$a_2^{(1)}$、$a_3^{(3)}$も同様にして求めることができますがこれら第１層目のニューロン$a_i^{(1)}$は以下のように行列の内積でまとめて表すことができます。
<div align=center>
$$
\begin{align}
        A^{(1)} = XW^{(1)} + B^{(1)} \quad &&(6.2)\\
\end{align}
$$
</div>

ただし、$A^{(1)}$ 、$X$、$W^{(1)}$、$B^{(1)}$はそれぞれ以下のようになります。
<div align=center>
$$
\begin{align}
        A^{(1)} &= \left( a_1^{(1)} a_2^{(1)} a_3^{(1)} \right) \quad &&(6.3)\\
        X &= \left( x_1 x_2 \right) \quad &&(6.4)\\
        W^{(1)} &= \left(
            \begin{array}{ccc}
                w_{11}^{(1)} & w_{21}^{(1)} & w_{31}^{(1)}  \\
                w_{12}^{(1)} & w_{22}^{(1)} & w_{32}^{(1)} 
            \end{array}
        \right) \quad &&(6.4)\\
        B^{(1)} &= \left( b_1^{(1)} b_2^{(1)} b_3^{(1)} \right) \quad &&(6.5)
\end{align}
$$
</div>

ここまでをnumpyで実装すると以下のようなコードになります。なお入力$x$は４章データの準備・説明にて抽出したirisデータの先頭サンプルを代入します。サンプルの正解ラベルはsetosa(=0)となっています。

入力する先頭サンプルのデータを確認しましょう。

In [0]:
print(f'花の種類:{Y[0]}')
print(f'ガクの長さ(x1):{X[0][0]}')
print(f'花弁の長さ(x2):{X[0][1]}')

花の種類:setosa
ガクの長さ(x1):5.1
花弁の長さ(x2):1.4


以下のコードで、$X$、$W^{(1)}$、$B^{(1)}$をセットします。

In [0]:
x = X[0]
w1 = np.array([[-0.9,-0.9,-0.5],[0.9,0.5,1.8]])
b1 = np.array([0,0,-0.4])

ここでセットした重みとバイアスは、既に学習済みのパラメータとなります。本来は重みとバイアスの初期値には乱数が与えられ、学習を繰り返すたびに良い分類結果を出力する値へと更新されていきます。本テキストではニューラルネットワークの学習については省略します。

numpyでは以下のコードのように、簡単に行列計算を行うことができます。

[-3.33 -3.89 -0.43]


以上で、$A^{(1)}$を求めることができました。

## ReLU関数
図5.3で示したように、入力に重みを掛け合わせバイアスを足し終えた各ニューロン$a$は、活性化関数と呼ばれる関数によって変換が行われます。活性化関数にはReLU関数、シグモイド関数、tanh関数などが用いられていますが、今回はReLU関数を用います。ReLU関数をグラフで表すと以下の図6.3のようになります。

<div align=center>
![代替テキスト](https://cdn-images-1.medium.com/max/1600/1*DfMRHwxY1gyyDmrIAd-gjQ.png)<br>
図6.3　Relu関数
</div><br>



ReLU関数を数式で表すと以下のようになります。
<div align=center>
$$
\begin{align}
        ReLU(x) &= max(0,x) = 
        \begin{cases}
            x(x\geq 0)\\
            0(x<0)
        \end{cases}\quad &&(6.6)\\
\end{align}
$$
</div>
活性化関数の役割、活性化関数それぞれの特徴については本テキストでは省略します。

ReLU関数を以下のコードによって実装します。

In [0]:
def relu(x):
    return np.maximum(0, x)

先ほど導出した$a_1^{(1)}$をReLU関数を用いて変換します。ここまでの伝達を図で表すと以下のようになります。活性化関数は$h()$として表します。



<div align=center>
![代替テキスト](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550177345.png)<br>
図6.3　入力層から第１層目の１番目のニューロン$z_1^{(1)}$への伝達
</div><br>


以下に、ReLU関数を用いて$A^{(1)}$の変換を行うコードを示します。

[0. 0. 0.]


これで第１層目の出力$Z^{(1)}$を求めることができました。

## 第１層目⇨第２層目
次に、隠れ層である第１層目から第２層目への伝達の実装を行います。第１層目から第２層目の１番目のニューロン$z_1^{(2)}$への伝達を以下の図6.4で表します。


<div align=center>
![代替テキスト](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550319280.png)<br>
図6.4　第１層目から第２層目の１番目のニューロン$z_1^{(2)}$への伝達
</div><br>

以下のコードで、$W^{(2)}$、$B^{(2)}$をセットします。

In [0]:
w2 = np.array([[0.9,-0.1],[0.7,0.6],[-0.7,1.4]])
b2 = np.array([0,-0.5])

第１層目⇨第２層目と同様に、$A^{(2)}$を行列の内積で導出します。

[ 0.  -0.5]


次に第１層目と同様に、ReLU関数によって$A^{(2)}$を$Z^{(2)}$へ変換します。

[0. 0.]


以上で、第１層目から第２層目への伝達は完了です。

## 第２層目⇨出力層
最後に、隠れ層である第２層目から出力層への伝達の実装を行います。第２層目から出力層１番目のニューロン$a_1^{(3)}$への伝達を、以下の図6.5で表します。


<div align=center>
![代替テキスト](http://wiki.nepp.tokyo:8080/uploads/images/drawio/2019-02-Feb/Drawing-Kushiro-Taichi-1550320909.png)<br>
図6.4　第２層目からの出力層１番目のニューロン$a_1^{(3)}$への伝達
</div><br>

以下のコードで、$W^{(3)}$、$B^{(3)}$をセットします。

In [0]:
w3 = np.array([[-0.8,0.1],[-0.6,1.2]])
b3 = np.array([1.8,-1.8])

次に$A^{(3)}$を行列の内積で導出します。

[ 1.8 -1.8]


以上で、$A^{(3)}$を求めることができました。

## ソフトマックス関数
最後に、活性化関数によって$A^{(3)}$から$Y$への変換を行います。しかしここではReLU関数は用いず、ソフトマックス関数と呼ばれる活性化関数を用います。  
最終的な出力層において、それぞれのニューロンが何番目であるかを$i$番目と表す場合、ソフトマックス関数を用いて出力$y_i$を求める際は、以下のような計算式となります。なお、$n$は出力層におけるニューロンの個数とします。

<div align=center>
$$
\begin{align}
        y_i & = \frac{exp(a_i)}{\sum_{k=1}^n exp(a_k)}\quad &&(6.7)\\
\end{align}
$$
</div>

例として、$A=\left( a_1,a_2\right) = \left( 1.2,1.8\right)$となるような入力$A$が与えらたとき、ソフトマックス関数は$Y=\left( y_1,y_2\right) = \left( 0.4,0.6\right)$のように、$\sum_{i=1}^n y_i = 1$となる値を出力します。

よって、ニューラルネットワークの出力をあたかもサンプルがどの分類に属するかの確率のように表すことができます。

ソフトマックス関数はnumpyを用いると以下のコードで実装できます。ここでは、コードの説明は省略します。

In [0]:
def softmax(a):
    c=np.max(a)
    exp_a=np.exp(a-c)
    sum_exp_a=np.sum(exp_a)
    y=exp_a/sum_exp_a
    
    return y

ソフトマックス関数を用いて、$A^{(3)}$から出力$Y$への変換を行います。

[0.97340301 0.02659699]


以上の出力$y$は、与えられた入力$X$のデータを持つサンプルがsetosaである確率が約$97.3\%$、versicolorである確率が約$2.7\%$と捉えることができます。抽出したサンプルの先頭である入力$X$はsetosaのサンプルであったため、高い分類結果を出力することができました。

## 実装まとめ
これまでの流れを関数にまとめたものが以下のコードになります。

In [0]:
def forword(x):
    w1 = np.array([[-0.9,-0.9,-0.5],[0.9,0.5,1.8]])
    b1 = np.array([0,0,-0.4])
    w2 = np.array([[0.9,-0.1],[0.7,0.6],[-0.7,1.4]])
    b2 = np.array([0,-0.5])
    w3 = np.array([[-0.8,0.1],[-0.6,1.2]])
    b3 = np.array([1.8,-1.8])
    
    return y

作成した関数を用いて、抽出した全てのサンプル100個の分類を行います。

In [0]:
for i in range(100):
    x = np.array(X[i])
    y=forword(x)
    print('花の種類：', Y[i])
    print(f'ガクの長さ:{X[i][0]}')
    print(f'花弁の長さ:{X[1][0]}')
    print(f'予測割合： setosa -> {round(y[0],3)} versicolor -> {round(y[1],3)}',)
    print('予測結果：',iris_name[np.argmax(y)])
    print('---------------------------')

花の種類： setosa
ガクの長さ:5.1
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:4.9
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:4.7
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:4.6
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:5.0
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:5.4
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:4.6
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:5.0
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.027
予測結果： setosa
---------------------------
花の種類： setosa
ガクの長さ:4.4
花弁の長さ:4.9
予測割合： setosa -> 0.973 versicolor -> 0.0

以上で、irisデータセットの花の分類を行うニューラルネットワークを実装することができました。

# 参考文献
[ゼロから作るディープラーニング](https://www.amazon.co.jp/dp/4873117585/ref=asc_df_48731175852587512/?tag=jpgo-22&creative=9303&creativeASIN=4873117585&linkCode=df0&hvadid=295686767484&hvpos=1o1&hvnetw=g&hvrand=1462346276493694429&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=1009298&hvtargid=pla-526224399321&th=1&psc=1)

[PRMLの線形回帰モデル（線形基底関数モデル）](https://www.slideshare.net/yasunoriozaki12/prml-29439402)

[単回帰分析 | 統計学の時間 | 統計WEB](https://bellcurve.jp/statistics/course/9700.html)

[[機械学習] iris データセットを用いて scikit-learn の様々な分類アルゴリズムを試してみた](https://qiita.com/ao_log/items/fe9bd42fd249c2a7ee7a)

[NumPyでニューラルネットワークを実装してみる 実装編](https://deepage.net/features/numpy-neuralnetwork-3.html)